# Testing for E1 - COGNITION
fistly define the UUID of the experiment to use

In [1]:
experimentId="352338b0-f723-421f-a69e-e2afb40205ed"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load cognition

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath)

In [5]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,labelerId,type,samplerate,startInd,stopInd
0,0001,C3,1938.132297,1939.202326,1.070029,2,0001,spindle,200,387626,387840
1,0001,C3,2047.217898,2048.112834,0.894936,2,0001,spindle,200,409444,409623
2,0001,C3,2170.856032,2171.634237,0.778205,3,0001,spindle,200,434171,434327
3,0001,C3,2219.766538,2220.894943,1.128405,3,0001,spindle,200,443953,444179
4,0001,C3,2263.482492,2264.357974,0.875482,3,0001,spindle,200,452696,452872


In [6]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Load experiment results

In [7]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [8]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001,16b9b403-5b57-4f9b-b9b4-dd56240acf77,0.008985
1,0001,E1,[0001],"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002,07e6bbaa-2aa2-4df4-8e5f-28bc716dc817,0.005890
2,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003,53f1c280-5951-4042-852e-00443c71b12e,0.005867
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004,e8ce0b00-6a48-4362-bc01-501a4996914c,0.008876
4,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005,90ab0dfb-22de-4127-bcbc-ea6d81cf7540,0.005867
5,0001,E1,[0001],"[0003, 0004, 0005, 0007, 0008, 0009]","[0001, 0002]",0006,fd90c505-fd19-4b2a-bbb7-a88529c36d7c,0.009365
6,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0008]","[0006, 0009]",0007,5278edde-a3c5-4161-9613-f4be8bd14fc0,0.009711
7,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0006]","[0009, 0007]",0008,81a44421-f8a4-48c9-9dba-9aa04ab97659,0.010361
8,0001,E1,[0001],"[0001, 0003, 0004, 0005, 0006, 0008]","[0007, 0002]",0009,f1d48762-3da9-4fe4-920f-943298435e53,0.009431


In [9]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                                  \
                        count      mean      std       min      25%       50%   
criteriumName                                                                   
E1                        9.0  0.008261  0.00184  0.005867  0.00589  0.008985   

                                   
                    75%       max  
criteriumName                      
E1             0.009431  0.010361

In [10]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [11]:
hyperClose=0.25
hyperDuration=0.3
hyperThres=0.4
hyperDepth=30

## Testing with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [12]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001,16b9b403-5b57-4f9b-b9b4-dd56240acf77,0.008985
1,0001,E1,[0001],"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002,07e6bbaa-2aa2-4df4-8e5f-28bc716dc817,0.005890
2,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003,53f1c280-5951-4042-852e-00443c71b12e,0.005867
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004,e8ce0b00-6a48-4362-bc01-501a4996914c,0.008876
4,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005,90ab0dfb-22de-4127-bcbc-ea6d81cf7540,0.005867
5,0001,E1,[0001],"[0003, 0004, 0005, 0007, 0008, 0009]","[0001, 0002]",0006,fd90c505-fd19-4b2a-bbb7-a88529c36d7c,0.009365
6,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0008]","[0006, 0009]",0007,5278edde-a3c5-4161-9613-f4be8bd14fc0,0.009711
7,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0006]","[0009, 0007]",0008,81a44421-f8a4-48c9-9dba-9aa04ab97659,0.010361
8,0001,E1,[0001],"[0001, 0003, 0004, 0005, 0006, 0008]","[0007, 0002]",0009,f1d48762-3da9-4fe4-920f-943298435e53,0.009431


In [13]:
#threshold for the metric by event
masterIoU=0.2

In [14]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [15]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 9
*************************
2 of 9
*************************
3 of 9
*************************
4 of 9
*************************
5 of 9
*************************
6 of 9
*************************
7 of 9
*************************
8 of 9
*************************
9 of 9


In [16]:
dumpPickle('experimentModelsTest_E1_temp.pkl',experimentModels)

In [17]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [18]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [19]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [20]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [21]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [22]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.114537 NaN  0.114537  0.114537  0.114537            1.0   
0002     1.0  0.665796 NaN  0.665796  0.665796  0.665796            1.0   
0003     1.0  0.871652 NaN  0.871652  0.871652  0.871652            1.0   
0004     1.0  0.767490 NaN  0.767490  0.767490  0.767490            1.0   
0005     1.0  0.621762 NaN  0.621762  0.621762  0.621762            1.0   
0006     1.0  0.692473 NaN  0.692473  0.692473  0.692473            1.0   
0007     1.0  0.687419 NaN  0.687419  0.687419  0.687419            1.0   
0008     1.0  0.517647 NaN  0.517647  0.517647  0.517647            1.0   
0009     1.0  0.152672 NaN  0.152672  0.152672  0.152672            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  1.000000 NaN  1.000000  1.000000  1.000000         1.0  0.060748 NaN   
0002  0.895105 NaN  0.895105  0.895105  0.895105         1.0  0.529167 NaN   
0003  0.829268 NaN  0.829268  0.829268  0.829268         1.0  0.918728 NaN   
0004  0.674503 NaN  0.674503  0.674503  0.674503         1.0  0.890215 NaN   
0005  0.462250 NaN  0.462250  0.462250  0.462250         1.0  0.949367 NaN   
0006  0.673640 NaN  0.673640  0.673640  0.673640         1.0  0.712389 NaN   
0007  0.664160 NaN  0.664160  0.664160  0.664160         1.0  0.712366 NaN   
0008  0.578947 NaN  0.578947  0.578947  0.578947         1.0  0.468085 NaN   
0009  0.344828 NaN  0.344828  0.344828  0.344828         1.0  0.098039 NaN   

                                    
           min       50%       max  
test                                
0001  0.060748  0.060748  0.060748  
0002  0.529167  0.529167  0.529167  
0003  0.918728  0.918728  0.918728  
0004  0.890215  0.890215  0.890215  
0005  0.949367  0.949367  0.949367  
0006  0.712389  0.712389  0.712389  
0007  0.712366  0.712366  0.712366  
0008  0.468085  0.468085  0.468085  
0009  0.098039  0.098039  0.098039

by-event metrics

In [23]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.114537,1.000000,0.060748
1,0002,0.665796,0.895105,0.529167
2,0003,0.871652,0.829268,0.918728
3,0004,0.767490,0.674503,0.890215
4,0005,0.621762,0.462250,0.949367
5,0006,0.692473,0.673640,0.712389
6,0007,0.687419,0.664160,0.712366
7,0008,0.517647,0.578947,0.468085
8,0009,0.152672,0.344828,0.098039


In [24]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.111333e+31
eventF1           5.657164e-01
eventPrecision    6.803001e-01
eventRecall       5.932337e-01
dtype: float64

In [25]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_20336\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.263350
eventPrecision    0.206531
eventRecall       0.335017
dtype: float64

by-sample metrics

In [28]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         1.111333e+31
f1           5.055543e-01
precision    6.557885e-01
recall       5.070047e-01
dtype: float64

In [29]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_20336\634953098.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.250518
precision    0.198658
recall       0.301683
dtype: float64

In [26]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [27]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [30]:
np.setdiff1d(experimentModels.test,['0001','0009'])

array(['0002', '0003', '0004', '0005', '0006', '0007', '0008'],
      dtype=object)

In [31]:
experimentModels[experimentModels.test.isin(np.setdiff1d(experimentModels.test,['0001','0009']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              2.857571e+23
eventF1           6.891769e-01
eventPrecision    6.825534e-01
eventRecall       7.400452e-01
dtype: float64

In [32]:
experimentModels[experimentModels.test.isin(np.setdiff1d(experimentModels.test,['0001','0009']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_20336\1948016846.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.111024
eventPrecision    0.145175
eventRecall       0.190725
dtype: float64

by-sample

In [35]:
experimentModels[experimentModels.test.isin(np.setdiff1d(experimentModels.test,['0001','0009']))][['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         2.857571e+23
f1           6.228170e-01
precision    6.539085e-01
recall       6.361175e-01
dtype: float64

In [34]:
experimentModels[experimentModels.test.isin(np.setdiff1d(experimentModels.test,['0001','0009']))][['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_20336\1417307937.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.105806
precision    0.135251
recall       0.183525
dtype: float64